In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
class DT():
  def partition(self, x):
      """
      Partition the column vector x into subsets indexed by its unique values (v1, ... vk)

      Returns a dictionary of the form
      { v1: indices of x == v1,
        v2: indices of x == v2,
        ...
        vk: indices of x == vk }, where [v1, ... vk] are all the unique values in the vector z.
      """
      unique_values = set(x)

      # Create an empty dictionary to store the indices of each unique value
      indices_dict = {}

      # Iterate through the unique values and find the indices where x equals each value
      for value in unique_values:
          indices = [i for i in range(len(x)) if x[i] == value]
          indices_dict[value] = indices
      return indices_dict
      


  def entropy(self, y):
      """
      Compute the self.entropy of a vector y by considering the counts of the unique values (v1, ... vk), in z

      Returns the self.entropy of z: H(z) = p(z=v1) log2(p(z=v1)) + ... + p(z=vk) log2(p(z=vk))
      """

      value_counts = {}
      for value in y:
          if value in value_counts:
              value_counts[value] += 1
          else:
              value_counts[value] = 1
      entropy = 0
      for count in value_counts.values():
          p = count / len(y)
          entropy -= p * np.log2(p)
      return entropy
      


  def mutual_information(self, x, y):
      """
      Compute the mutual information between a data column (x) and the labels (y). The data column is a single attribute
      over all the examples (n x 1). Mutual information is the difference between the self.entropy BEFORE the split set, and
      the weighted-average self.entropy of EACH possible split.

      Returns the mutual information: I(x, y) = H(y) - H(y | x)
      """
      entropy_y = self.entropy(y)
      value_indices = self.partition(x)
      entropy_y_given_x = 0

      for value in value_indices.keys():

          entries_for_pair = len(value_indices[value])
          sub_y = [y[i] for i in value_indices[value]]
          entropy_y_given_x += (entries_for_pair / len(x)) * self.entropy(sub_y)

      return entropy_y - entropy_y_given_x

      


  def id3(self, x, y, attribute_value_pairs=None, depth=0, max_depth=5):
      """
      Implements the classical ID3 algorithm given training data (x), training labels (y) and an array of
      attribute-value pairs to consider. This is a recursive algorithm that depends on three termination conditions
          1. If the entire set of labels (y) is pure (all y = only 0 or only 1), then return that label
          2. If the set of attribute-value pairs is empty (there is nothing to split on), then return the most common
            value of y (majority label)
          3. If the max_depth is reached (pre-pruning bias), then return the most common value of y (majority label)
      Otherwise the algorithm selects the next best attribute-value pair using INFORMATION GAIN as the splitting criterion
      and self.partitions the data set based on the values of that attribute before the next recursive call to ID3.

      The tree we learn is a BINARY tree, which means that every node has only two branches. The splitting criterion has
      to be chosen from among all possible attribute-value pairs. That is, for a problem with two features/attributes x1
      (taking values a, b, c) and x2 (taking values d, e), the initial attribute value pair list is a list of all pairs of
      attributes with their corresponding values:
      [(x1, a)
      (x1, b),
      (x1, c),
      (x2, d),
      (x2, e)]
      If we select (x2, d) as the best attribute-value pair, then the new decision node becomes: [ (x2 == d)? ] and
      the attribute-value pair (x2, d) is removed from the list of attribute_value_pairs.

      The tree is stored as a nested dictionary, where each entry is of the form
                      (attribute_index, attribute_value, True/False): subtree
      * The (attribute_index, attribute_value) determines the splitting criterion of the current node. For example, (4, 2)
      indicates that we test if (x4 == 2) at the current node.
      * The subtree itself can be nested dictionary, or a single label (leaf node).
      * Leaf nodes are (majority) class labelsX

      Returns a decision tree represented as a nested dictionary, for example
      {(4, 1, False):
          {(0, 1, False):
              {(1, 1, False): 1,
              (1, 1, True): 0},
          (0, 1, True):
              {(1, 1, False): 0,
              (1, 1, True): 1}},
      (4, 1, True): 1}
      """

      if np.sum(y) == len(y):
          return 1

      if np.sum(y) == 0:
          return 0

      if depth >= max_depth:
          return 0 if len(y) - np.sum(y) > np.sum(y) else 1

      if attribute_value_pairs == None:
          attribute_value_pairs = dict()

          for col_idx in range(len(x[0])):
              value_idx_dict = self.partition(x[:, col_idx])
              for value in value_idx_dict.keys():
                  x_bar = [0]*len(x[:, col_idx])
                  for change_idx in range(len(x[:, col_idx])):
                      if change_idx in value_idx_dict[value]:
                          x_bar[change_idx] = value

                  attribute_value_pairs[(col_idx, value)
                                        ] = self.mutual_information(x_bar, y)

      max_pair, _ = max(attribute_value_pairs.items(), key=lambda x: x[1])

      sol_dict = dict()
      x_new = np.copy(x)

      attribute_value_pairs.__delitem__(max_pair)

      value_idx_dict = self.partition(x[:, max_pair[0]])
      to_be_removed = [i for i in range(
          len(x[:, max_pair[0]])) if i in value_idx_dict[max_pair[1]]]
      x_new = np.delete(x, (to_be_removed), 0)
      y_new = np.delete(y, (to_be_removed), 0)

      sol_dict[(max_pair[0], max_pair[1], False)] = self.id3(
          x_new, y_new, None, depth+1, max_depth)

      value_idx_dict = self.partition(x[:, max_pair[0]])
      to_be_removed = [i for i in range(
          len(x[:, max_pair[0]])) if i not in value_idx_dict[max_pair[1]]]
      x_new = np.delete(x, (to_be_removed), 0)
      y_new = np.delete(y, (to_be_removed), 0)

      if (len(y_new) == 0):
          sol_dict[(max_pair[0], max_pair[1], True)] = 0 if len(
              y_new) - np.sum(y_new) > np.sum(y_new) else 1
      else:
          sol_dict[(max_pair[0], max_pair[1], True)] = self.id3(
              x_new, y_new, None, depth+1, max_depth)

      return sol_dict

      


  def predict_example(self, x, tree):
      """
      Predicts the classification label for a single example x using tree by recursively descending the tree until
      a label/leaf node is reached.

      Returns the predicted label of x according to tree
      """
      y_pred = -1

      while (True):
          if tree == 1:
              return 1
          if tree == 0:
              return 0

          branch1, branch2 = tree.keys()
          feature = branch1[0]
          value = branch1[1]

          if (x[feature] == value):

              tree = tree[(feature, value, True)]
          else:
              tree = tree[(feature, value, False)]

      


  def compute_error(self, y_true, y_pred):
      """
      Computes the average error between the true labels (y_true) and the predicted labels (y_pred)

      Returns the error = (1/n) * sum(y_true != y_pred)
      """

      n = len(y_true)
      error = 0
      for i in range(n):
          if (y_true[i] != y_pred[i]):
              error = error + 1

      return error / n
      


  def confusion_matrix(self, y_true, y_pred):
      tp = 0
      tn = 0
      fp = 0
      fn = 0
      for i in range(len(y_true)):
          if y_true[i] == 1 and y_pred[i] == 1:
              tp += 1
          if y_true[i] == 1 and y_pred[i] == 0:
              fn += 1
          if y_true[i] == 0 and y_pred[i] == 0:
              tn += 1
          if y_true[i] == 0 and y_pred[i] == 1:
              fp += 1
      return [[tp, fn], [fp, tn]]


  def visualize(self, tree, depth=0):
      """
      Pretty prints (kinda ugly, but hey, it's better than nothing) the decision tree to the console. Use print(tree) to
      print the raw nested dictionary representation.
      DO NOT MODIFY THIS FUNCTION!
      """

      if depth == 0:
          print('TREE')

      for index, split_criterion in enumerate(tree):
          sub_trees = tree[split_criterion]

          # Print the current node: split criterion
          print('|\t' * depth, end='')
          print(
              '+-- [SPLIT: x{0} = {1}]'.format(split_criterion[0], split_criterion[1]))

          # Print the children
          if type(sub_trees) is dict:
              self.visualize(sub_trees, depth + 1)
          else:
              print('|\t' * (depth + 1), end='')
              print('+-- [LABEL = {0}]'.format(sub_trees))

In [22]:

# Load the training data
df = pd.read_csv('./adult.data', header= None)
label_encoder = preprocessing.LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

# remove the continuous column
del df[2]


In [23]:
df.head()

,0,1,3,4,5,6,7,8,9,10,11,12,13,14
0,22,7,9,12,4,1,1,4,1,25,0,39,39,0
1,33,6,9,12,2,4,0,4,1,0,0,12,39,0
2,21,4,11,8,0,6,1,4,1,0,0,39,39,0
3,36,4,1,6,2,6,0,2,1,0,0,39,39,0
4,11,4,9,12,2,10,5,2,0,0,0,39,5,0


In [34]:

dataset = df.to_numpy()[:10000]

Y = dataset[:, -1]
X = dataset[:, 0:-1]

Xtrn, Xtst, ytrn, ytst = train_test_split(
    X, Y, test_size=0.3, random_state=42)

In [35]:
decision_tree_obj = DT()
decision_tree = decision_tree_obj.id3(Xtrn, ytrn, max_depth=3)
y_pred_train = [decision_tree_obj.predict_example(x, decision_tree) for x in Xtrn]
trn_err = decision_tree_obj.compute_error(ytrn, y_pred_train)

y_pred_tst = [decision_tree_obj.predict_example(x, decision_tree) for x in Xtst]
tst_err = decision_tree_obj.compute_error(ytst, y_pred_tst)

decision_tree_obj.visualize(decision_tree)

mst = decision_tree_obj.confusion_matrix(ytst, y_pred_tst)

print('Confusion matrix:', mst)
print('Test Error = {0:4.2f}%.'.format(tst_err * 100))


DTclf = DecisionTreeClassifier(max_depth= 3)
DTclf = DTclf.fit(Xtrn, ytrn)
prediction = DTclf.predict(Xtst)
tst_err_sk = decision_tree_obj.compute_error(ytst, prediction)
mst_sk = decision_tree_obj.confusion_matrix(ytst, prediction)

print('Confusion matrix of sk:', mst_sk)
print('Scikit-learns’s test Error = {0:4.2f}%.'.format(tst_err_sk * 100))
export_graphviz(DTclf,
                out_file='tree.dot',
                )

TREE
+-- [SPLIT: x4 = 2]
|	+-- [SPLIT: x6 = 3]
|	|	+-- [SPLIT: x9 = 97]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x9 = 97]
|	|	|	+-- [LABEL = 1]
|	+-- [SPLIT: x6 = 3]
|	|	+-- [SPLIT: x1 = 6]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x1 = 6]
|	|	|	+-- [LABEL = 0]
+-- [SPLIT: x4 = 2]
|	+-- [SPLIT: x5 = 4]
|	|	+-- [SPLIT: x5 = 10]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x5 = 10]
|	|	|	+-- [LABEL = 1]
|	+-- [SPLIT: x5 = 4]
|	|	+-- [SPLIT: x2 = 12]
|	|	|	+-- [LABEL = 1]
|	|	+-- [SPLIT: x2 = 12]
|	|	|	+-- [LABEL = 1]
Confusion matrix: [[298, 420], [134, 2148]]
Test Error = 18.47%.
Confusion matrix of sk: [[341, 377], [107, 2175]]
Scikit-learns’s test Error = 16.13%.
